# Tests for `predictor.utils`

## How to use:

Check your current kernel, if it's not the project's venv, change it:

* Click the top right button to change the kernel to your project's venv (e.g. `'mobilidade_rio_api' (Python 3.10.8)` ([learn more](https://code.visualstudio.com/docs/python/jupyter-support-py#_connect-to-a-remote-jupyter-server)).

# Code

Imports

In [1]:
import init_django
import pandas as pd
# django
from mobilidade_rio.pontos.models import *
from mobilidade_rio.predictor.models import *
# query
from django.db import connection
cur = connection.cursor() 
db_name = str(connection.settings_dict['NAME'])
[db_name.rsplit(x, 1)[-1] for x in "\\/."]
# environment variables
import os
print("Env loaded:", bool(os.environ.get('API_REALTIME')))

db_name: postgres
Env loaded: True


test task

In [3]:
from mobilidade_rio.config_django_q import tasks
# print(len(Prediction.objects.all()))
tasks.generate_prediction()
# print(len(Prediction.objects.all()))

GR 2.1 df_realtime:    codigo    placa trip_short_name   latitude  longitude            dataHora  \
0  901008                       51 -22.964575 -43.392185 2023-02-06 10:24:29   
1  901011  RIV8A74              53 -22.884430 -43.399342 2023-02-06 10:24:17   
2  901018                       51 -22.869043 -43.413025 2023-02-06 10:24:13   
3  901012                       22 -23.006863 -43.312332 2023-02-06 10:24:29   
4  901014                       53 -23.000908 -43.367333 2023-02-06 09:58:18   

   velocidade id_migracao_trajeto  direction_id  \
0         0.0                               1.0   
1         0.0                               0.0   
2        25.9                               0.0   
3         0.0                               1.0   
4         0.0                               1.0   

                                         trajeto  hodometro  
0      51 - SENTIDO V. MILITAR (PARADOR) [VOLTA]  45642.262  
1         53 - SENTIDO ALVORADA (EXPRESSO) [IDA]  24229.043  
2     

predictorView

In [49]:
import django.db.models as dm
queryset = Prediction.objects.all().order_by(
    "dataHora", "trip_short_name", "stop_id")

# filtrar por trip_name (trip_short_name)
trip_name = "51"
if trip_name:
    trip_name = trip_name.split(",")
    queryset = queryset.filter(trip_short_name__in=trip_name)

# calcular tempo de viagem até stop_id_destiny
stop_id_destiny = "533135204000271"
if stop_id_destiny:
    stop_id_destiny = stop_id_destiny.split(",")
    # TODO: 1. Ordenar por stop_sequence
    # queryset = queryset.order_by("pk")
    # 2. Selecionar da linha que contém id_veiculo
    veiculos = queryset.filter(id_veiculo__isnull=False, arrival_time__isnull=False)
    # 3. Para cada veículo, selecionar paradas posteriores e somar arrival_time
    df_veiculos = pd.DataFrame.from_records(veiculos.values())
    for i, veiculo in enumerate(veiculos):
        posteriores = queryset.filter(
            pk__gt=veiculo.pk, trip_id=veiculo.trip_id, arrival_time__isnull=False)
        posteriores_sum = list(posteriores.aggregate(dm.Sum("arrival_time")).values())[0]
        # print("VA", veiculo.arrival_time, "PS", posteriores_sum)
        arrival_time__new = veiculo.arrival_time + posteriores_sum
        veiculo.arrival_time = arrival_time__new
        # queryset can't be changed directly
        df_veiculos.loc[i, "arrival_time"] = arrival_time__new

        # ! Este método alteraria o modelo original
        # veiculos.filter(pk=veiculo.pk).update(arrival_time=arrival_time__new)
        # veiculo.update(arrival_time=veiculo.arrival_time + posteriores_sum)

    # queryset = veiculos

    queryset = [dict(zip([col for col in df_veiculos], row)) for row in df_veiculos.values]
    # queryset = df_veiculos.to_dict("records")
    

# from original queryset
# df = pd.DataFrame.from_records(queryset.values())
# from queryset as list of dicts
df = pd.DataFrame(queryset)
print("O tipo do campo arrival_time é o mesmo na queryset original e na nova")
display(df.head())
display(df.info())
# compare with original field
df = pd.DataFrame.from_records(Prediction.objects.filter(pk__in=[14]).values())
display(df.head())
display(df.info())


O tipo do campo arrival_time é o mesmo na queryset original e na nova


,id,trip_id,stop_id,id_veiculo,latitude,longitude,dataHora,trip_short_name,direction_id,service_id,arrival_time
0,14,d8619871-2c03-4309-9041-177097931b07,416115204000371,901018,-22.9632,-43.39368,2023-02-01 02:19:18.647544,51,1,U,0 days 00:00:09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   id               1 non-null      int64          
 1   trip_id          1 non-null      object         
 2   stop_id          1 non-null      object         
 3   id_veiculo       1 non-null      object         
 4   latitude         1 non-null      float64        
 5   longitude        1 non-null      float64        
 6   dataHora         1 non-null      object         
 7   trip_short_name  1 non-null      object         
 8   direction_id     1 non-null      object         
 9   service_id       1 non-null      object         
 10  arrival_time     1 non-null      timedelta64[ns]
dtypes: float64(2), int64(1), object(7), timedelta64[ns](1)
memory usage: 216.0+ bytes


None

,id,trip_id,stop_id,id_veiculo,latitude,longitude,dataHora,trip_short_name,direction_id,service_id,arrival_time
0,14,d8619871-2c03-4309-9041-177097931b07,416115204000371,901018,-22.9632,-43.39368,2023-02-01 02:19:18.647544,51,1,U,0 days 00:00:01


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   id               1 non-null      int64          
 1   trip_id          1 non-null      object         
 2   stop_id          1 non-null      object         
 3   id_veiculo       1 non-null      object         
 4   latitude         1 non-null      float64        
 5   longitude        1 non-null      float64        
 6   dataHora         1 non-null      object         
 7   trip_short_name  1 non-null      object         
 8   direction_id     1 non-null      object         
 9   service_id       1 non-null      object         
 10  arrival_time     1 non-null      timedelta64[ns]
dtypes: float64(2), int64(1), object(7), timedelta64[ns](1)
memory usage: 216.0+ bytes


None

# Tests

Maybe feature: set types from db

In [2]:
# map
postgres_pandas_types = {
    'integer': 'int64',
    'character varying': 'str',
    'double precision': 'float64',
}

# get columns and types
query_col_types = f"""
select column_name as col, data_type as type
from information_schema.columns
where table_name = '{Prediction._meta.db_table}'
"""
cur.execute(query_col_types)
cols = [desc[0] for desc in cur.description]
fields = [dict(zip(cols, row)) for row in cur.fetchall()]

# convert types
fields_converted = {x['col']: postgres_pandas_types[x['type']] for x in fields}


# test
# get first 3 rows
df = pd.DataFrame.from_records(Prediction.objects.all()[:3].values())
df = df.astype(fields_converted)
# change direction_id[0]
df.at[0, 'direction_id'] = pd.NA
df.at[0, 'id'] = 6666
# convert and clear
df = df.replace({pd.NA: None})
df = df.replace({"<NA>": None})
# df = df.astype(str)
df.head(3)
# insert first row
Prediction.objects.create(**df.iloc[0].to_dict())

KeyError: 'time without time zone'

Add df to django ORM

In [3]:
import hashlib

Agency.objects.all().delete()
# create df based on model
new_agency = Agency(
    agency_name = 'a1',
)
df_agency = pd.DataFrame([new_agency.__dict__])
df_agency = df_agency.drop(['_state'], axis=1)
# set random pk
random_md5 = hashlib.md5(os.urandom(128)).hexdigest()
df_agency['agency_id'] = random_md5

# add 3 random items
for i in range(3):
    random_md5 = hashlib.md5(os.urandom(128)).hexdigest()
    # concat with new values
    df_agency = pd.concat([df_agency, pd.DataFrame([{'agency_id': random_md5}])], ignore_index=True)

display(df_agency)
for row in df_agency.to_dict('records'):
    print("row",row)
#     Agency.objects.create(**row)

# bulk create
Agency.objects.bulk_create([Agency(**row) for row in df_agency.to_dict('records')])

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_branding_url,agency_fare_url,agency_email
0,999427adac028028a36ac4210d42a959,a1,,,None,None,None,None,None
1,35f115ba54f89222da2cc0d1a3074d30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3e4840d776769235ed1349561695448e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,fbf97e9c5f8f75a3203efe06bb447065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


row {'agency_id': '999427adac028028a36ac4210d42a959', 'agency_name': 'a1', 'agency_url': '', 'agency_timezone': '', 'agency_lang': None, 'agency_phone': None, 'agency_branding_url': None, 'agency_fare_url': None, 'agency_email': None}
row {'agency_id': '35f115ba54f89222da2cc0d1a3074d30', 'agency_name': nan, 'agency_url': nan, 'agency_timezone': nan, 'agency_lang': nan, 'agency_phone': nan, 'agency_branding_url': nan, 'agency_fare_url': nan, 'agency_email': nan}
row {'agency_id': '3e4840d776769235ed1349561695448e', 'agency_name': nan, 'agency_url': nan, 'agency_timezone': nan, 'agency_lang': nan, 'agency_phone': nan, 'agency_branding_url': nan, 'agency_fare_url': nan, 'agency_email': nan}
row {'agency_id': 'fbf97e9c5f8f75a3203efe06bb447065', 'agency_name': nan, 'agency_url': nan, 'agency_timezone': nan, 'agency_lang': nan, 'agency_phone': nan, 'agency_branding_url': nan, 'agency_fare_url': nan, 'agency_email': nan}


[<Agency: Agency object (999427adac028028a36ac4210d42a959)>,
 <Agency: Agency object (35f115ba54f89222da2cc0d1a3074d30)>,
 <Agency: Agency object (3e4840d776769235ed1349561695448e)>,
 <Agency: Agency object (fbf97e9c5f8f75a3203efe06bb447065)>]

In [8]:
queryset = Agency.objects.all()
# load df
df = pd.DataFrame.from_records(queryset.values())
df

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone,agency_branding_url,agency_fare_url,agency_email
0,999427adac028028a36ac4210d42a959,a1,,,None,None,None,None,None
1,35f115ba54f89222da2cc0d1a3074d30,nan,nan,nan,nan,nan,nan,nan,nan
2,3e4840d776769235ed1349561695448e,nan,nan,nan,nan,nan,nan,nan,nan
3,fbf97e9c5f8f75a3203efe06bb447065,nan,nan,nan,nan,nan,nan,nan,nan


queryset to df